In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
from langchain import OpenAI, PromptTemplate
from fpdf import FPDF
import creds as cr
# Initialize the OpenAI model with your API key
openai_api_key = cr.openai_api_key  # Replace with your actual OpenAI API key
llm = OpenAI(api_key=openai_api_key)

C:\Users\prata\Documents\YOJAI\Projects\RAG\.RAG_env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [2]:


class PDF(FPDF):
    def __init__(self, topic_text, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.topic_text = topic_text
        self.is_first_page = True

    def header(self):
        if self.is_first_page:
            # Center the topic text on the first page
            self.set_font("Arial", 'B', size=20)
            self.cell(0, 5, self.topic_text, ln=True, align='C')
            self.ln(10)  # Add space after the topic text on the first page
            self.is_first_page = False
        else:
            self.ln(10)  # Add space at the top of subsequent pages

    def footer(self):
        self.set_y(-15)  # Position from bottom
        self.set_font("Arial", 'I', size=8)
        self.cell(0, 10, f'Page {self.page_no()} / {{nb}}', 0, 0, 'C')

def replace_non_latin_chars(text):
    return text.encode('latin-1', 'replace').decode('latin-1')

def get_questions_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    questions = soup.find_all('li')
    extracted_questions = []
    for question in questions:
        question_str = str(question)
        question_str = re.sub(r'</?em>', ' ', question_str)
        question_soup = BeautifulSoup(question_str, 'html.parser')
        text = question_soup.get_text(strip=True)
        cleaned_text = text.replace("QuestionAnswer:", "").strip()
        extracted_questions.append(cleaned_text)
    return extracted_questions


def get_complete_answer(question, topic):
    max_context_length = 4097 - 256  # Model's max context length minus completion length
    prompt_template = PromptTemplate(input_variables=["question", "topic"], template="Topic: {topic}\nQ: {question}\nA:")
    topic_text = topic.split('-')[:-2]    
    topic_text = " ".join(str(item) for item in topic_text) 
    prompt = prompt_template.format(question=question, topic=topic_text)
    
    if len(prompt.split()) > max_context_length:
        prompt = ' '.join(prompt.split()[:max_context_length])
    
    complete_answer = ""
    while True:
        response = llm(prompt + complete_answer)
        new_text = response.strip()
        if new_text.endswith('.') or len(new_text.split()) < 5:
            complete_answer += new_text
            break
        else:
            complete_answer += new_text + ' '
    return complete_answer

def write_to_pdf(filename, topic_text, questions_and_answers):
    pdf = PDF(topic_text)
    pdf.alias_nb_pages()  # Add this to use {nb} in footer for total page count
    pdf.add_page()
    
    for question, answer in questions_and_answers:
        pdf.set_font("Arial", 'B', size=12)
        pdf.multi_cell(0, 5, replace_non_latin_chars(f"{question}"))
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 5, replace_non_latin_chars(f"{answer}"))
        pdf.ln(5)  # Add a new line for separation

    pdf.output(filename, 'F')

# Example usage
base_url = "https://devinterview.io"
url = "https://devinterview.io/questions/machine-learning-and-data-science/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Locate the <a> tags with the class 'TopicButton'
topics = soup.find_all('a', class_='TopicButton')

# Ensure the data folder exists
if not os.path.exists('data'):
    os.makedirs('data')

# Iterate through each topic link
for topic in topics:
    topic_text = topic.get_text(strip=True)
    topic_href = topic.get('href')
    
    new_url = base_url + topic_href
    print(f"Processing {new_url}")
    
    questions = get_questions_from_url(new_url)

    # Create a filename based on the topic URL
    topic_name = topic_href.split('/')[-2]
    
    filename = os.path.join('data', f"{topic_name}_with_answers.pdf")

    questions_and_answers = []
    for question in questions:
        answer = get_complete_answer(question, topic_name)
        questions_and_answers.append((question, answer))

    write_to_pdf(filename, topic_text, questions_and_answers)

    print(f"Questions and answers have been written to {filename}")


Processing https://devinterview.io/questions/machine-learning-and-data-science/anomaly-detection-interview-questions/


C:\Users\prata\Documents\YOJAI\Projects\RAG\.RAG_env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(

KeyboardInterrupt



In [127]:
# Individual topics data questions and answers
